## Introduction

Need to answer the questions:
- What is the machine learning problem you are trying to solve?
- Why does the problem matter?
- What could the results of your predictive model be used for?
- Why would we want to be able to predict the thing you're trying to preduct?
- Describe the dataset.


In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
%matplotlib inline

data = pd.read_csv("spotify-2023.csv")

### 1. Data Prep
#### 1.1 Data Cleaning
- Describe steps taken

In [2]:
# code here
#considering dropping apple/deezer data, one of these?
#take out artist names/track name
df = data.drop(['track_name', 'artist(s)_name'], axis=1)

# Fill in NaNs in the key column with C and the NaNs in in_shazam_charts with the mean
df['key'] = df['key'].fillna("C")
df['in_shazam_charts'] = df['in_shazam_charts'].str.replace(',', '', ).astype(float)
shazamMean = df['in_shazam_charts'].mean(skipna=True)
df['in_shazam_charts'] = df['in_shazam_charts'].fillna(shazamMean)

one_hot = pd.get_dummies(df['key'])
df = df.drop('key',axis = 1)
df = df.join(one_hot)

one_hot = pd.get_dummies(df['mode'])
df = df.drop('mode',axis = 1)
df = df.join(one_hot)

print(df.head())


   artist_count  released_year  released_month  released_day  \
0             2           2023               7            14   
1             1           2023               3            23   
2             1           2023               6            30   
3             1           2019               8            23   
4             1           2023               5            18   

   in_spotify_playlists  in_spotify_charts    streams  in_apple_playlists  \
0                   553                147  141381703                  43   
1                  1474                 48  133716286                  48   
2                  1397                113  140003974                  94   
3                  7858                100  800840817                 116   
4                  3133                 50  303236322                  84   

   in_apple_charts in_deezer_playlists  ...  C#  D  D#  E  F  F#  G  G#  \
0              263                  45  ...   0  0   0  0  0   0  0   0   
1 

#### 1.2 Data Exploration
- Describe steps taken

In [3]:
# code here
print(data.describe())

       artist_count  released_year  released_month  released_day  \
count    953.000000     953.000000      953.000000    953.000000   
mean       1.556139    2018.238195        6.033578     13.930745   
std        0.893044      11.116218        3.566435      9.201949   
min        1.000000    1930.000000        1.000000      1.000000   
25%        1.000000    2020.000000        3.000000      6.000000   
50%        1.000000    2022.000000        6.000000     13.000000   
75%        2.000000    2022.000000        9.000000     22.000000   
max        8.000000    2023.000000       12.000000     31.000000   

       in_spotify_playlists  in_spotify_charts  in_apple_playlists  \
count            953.000000         953.000000          953.000000   
mean            5200.124869          12.009444           67.812172   
std             7897.608990          19.575992           86.441493   
min               31.000000           0.000000            0.000000   
25%              875.000000          

#### 1.3 Feature Engineering
- Describe steps taken

In [4]:
# code here


#### 1.4 Additional ML Tasks
>  If your dataset requires very little cleaning and/or feature engineering, then the points in this area should be earned by doing something additional in another area!
>
> This could include more thorough/more in-depth data exploration; performing anomaly detection on the data; performing and evaluating various clustering algorithms; implementing more advance modeling techniques or algorithms (beyond the models taught in class); or other things beyond what is listed here.

### 2. Modeling
- Describe process and models tested

In [5]:
# code here

### 3. Results
- Describe results of final modeling